# Этап 8: AWQ (Activation-aware Weight Quantization) — Масштабирование

Статья [AWQ (arXiv:2308.07633)](https://arxiv.org/pdf/2308.07633) доказала, что не все веса одинаково важны. Веса, которые умножаются на большие значения активаций (outliers), вносят наибольшую ошибку при квантовании.

### Идея метода:
Вместо того чтобы оставлять важные веса в FP16 (что сложно аппаратно), мы будем их **масштабировать** вместе с активациями так, чтобы их было проще квантовать без потери точности.

In [ ]:
import torch
import torch.nn as nn
from src.model import GPTLanguageModel, device, get_batch, estimate_loss
import copy

model = GPTLanguageModel().to(device)
model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
model.eval()

def get_activations(model, layer, input_data):
    activations = []
    def hook(module, input, output):
        # Собираем абсолютные значения активаций
        activations.append(input[0].abs().detach())
    
    handle = layer.register_forward_hook(hook)
    model(input_data)
    handle.remove()
    return torch.cat(activations, dim=0).mean(dim=(0, 1)) # Усредняем по батчу и времени

## 1. Поиск оптимальных коэффициентов масштабирования

Формула трансформации:
$$W' = W \cdot s, \quad x' = x / s$$
Результат $y = Wx$ остается неизменным, но динамический диапазон весов $W$ меняется.

In [ ]:
def apply_awq_scaling(layer, acts, alpha=0.5):
    # Вычисляем scale на основе магнитуды активаций
    # s = acts^alpha / (W.max^ (1-alpha))
    w_max = layer.weight.abs().max(dim=0)[0]
    scales = acts.pow(alpha) / (w_max.pow(1-alpha) + 1e-6)
    scales = scales / (scales.max() * 0.5) # Нормализация
    
    # Применяем масштабирование к весам
    # Масштабируем столбцы весов
    layer.weight.data *= scales.view(1, -1)
    return scales

# Пример на первом слое MLP
xb, yb = get_batch('val')
target_layer = model.blocks[0].ffwd.net[0]
acts = get_activations(model, target_layer, xb)
scales = apply_awq_scaling(target_layer, acts)

print(f"Средний масштаб: {scales.mean().item():.4f}")
print("Масштабирование применено. Теперь квантование этого слоя будет более точным.")